# Load 'Liked Songs' Data from Spotify Account

### Grab user token

In [1]:
import spotipy.util as util
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd
import requests

username = '1266336804'
client_id = '40a1fa542f6d4637a0154eae9994a109'
client_secret = # insert secret id
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-library-read'

token = util.prompt_for_user_token(username=username,
                                  scope=scope,
                                  client_id=client_id,
                                  client_secret=client_secret,
                                  redirect_uri=redirect_uri)

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=token)

In [3]:
print(token)

BQCA9MwAad0L8tvgxzNNVRCY9DmuVlUx2e15iMCC6iNahIqyIaFjb2XnDmTBuZfV16UoRt4nWawFCLEvl7UQVnsvDH9Qy8_H8aCNbG7j48_6Zcp_QF0iwplfBYItslJJ5bfTI4_s56dPH7NaLsVQ9vozDTm2TxBCYA


### Create some functions that help with retrieving our data

In [4]:
# This grabs all of our 'Liked Songs'. Offset adjustments made to make up for get() only being able
#  to grab 50 songs at a time.

def get_saved_songs(token: str, offset: int) -> str:  # : str are annotations for what the variables mean,
    headers = {'Accept': 'application/json',                  # retrieve with f.__annotations__
                'Content-Type': 'application/json',
                'Authorization': f'Bearer ' + token}
    params = [('limit', 50), ('offset', offset * 50)]
    try:
        response = requests.get('https://api.spotify.com/v1/me/tracks',
                                headers=headers, params=params)
        json = response.json()
        return json
    except:
        return False

In [5]:
# checks if all values in an interating object are equal to each other
def all_equal(iterator):
    iterator = iter(iterator)
    try:
        first = next(iterator)
    except StopIteration:
        return True
    return all(first == x for x in iterator)

# checks if len of values for all keys are equal to each other in a dictionary
def dict_vals_len_equal(dict):
    val_len = []
    for key in dict:
        val_len.append(len(dict[key]))
    return all_equal(val_len)

Now utilize the functions above to create another function which retrieves all the songs under "Liked Songs" and places them in a dictionary. The dictionary will contain every "Liked" song and additional information associated with each song.

In [19]:
# get saved songs info in a dictionary
def get_saved_songs_dict(token: str):

    song_dict = {'id':[],
                 'name':[],
                 'album':[],
                 'artist':[],
                 'popularity':[],
                 'added_at':[]}
    
    offset = 0 # keeps track of how many songs we've retrieved so far (offset * 50)
    more_songs = True
    while(more_songs):
        
        saved_songs = get_saved_songs(token, offset)

        if len(saved_songs['items']) == 0:
            more_songs = False
        for item in saved_songs['items']:
            
             # track id
            track_id = item['track']['id']
            song_dict['id'].append(track_id)

            # added date/time
            track_added_at = item['added_at']
            song_dict['added_at'].append(track_added_at)

            # track name
            track_name = item['track']['name']
            song_dict['name'].append(track_name)

            # album name
            album_name = item['track']['album']['name']
            song_dict['album'].append(album_name)

            # artist
            artist = item['track']['artists'][0]['name']
            song_dict['artist'].append(artist)

            # popularity
            popularity = item['track']['popularity']
            song_dict['popularity'].append(popularity)
        offset += 1
    
    if dict_vals_len_equal(song_dict):
        return song_dict
    else:
        return "Not all values are equal lengths"

### Now convert "Liked Songs" to a dictionary

In [20]:
# convert liked songs to a dictionary
song_dict = get_saved_songs_dict(token)

### Grab Spotify's audio features for each song

In [21]:
features_dict = {'danceability':[],
            'energy':[],
            'key':[],
            'loudness':[],
            'mode':[],
            'speechiness':[],
            'acousticness':[],
            'instrumentalness':[],
            'liveness':[],
            'valence':[],
            'tempo':[],
            'time_signature':[]}

ids = song_dict['id']

for song_id in ids:
    song_features = sp.audio_features(song_id)[0]
    for feature in features_dict:
        features_dict[feature].append(song_features[feature])

### Concatenate the audio features dictionary to saved songs dictionary.

In [22]:
# concatenate saved songs dict with the features dict
saved_songs_dict = dict(song_dict, **features_dict)

### Convert dictionary to Pandas dataframe and save as a pickle file

In [23]:
# convert to panda dataframe and then save
saved_songs_df = pd.DataFrame.from_dict(saved_songs_dict)
saved_songs_df.head()

,id,name,album,artist,popularity,added_at,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,751gBcu62kORDelX7FV0mM,Thank You,No Angel,Dido,58,2021-08-27T05:32:10Z,0.722,0.595,1,-9.928,0,0.0415,0.284000,0.00054,0.0694,0.7380,79.981,4
1,7zKncUwLtKCMzwhEb3Jiqd,Overkill,Overkill,RIOT,48,2021-08-27T05:19:49Z,0.435,0.956,8,-1.369,0,0.1440,0.000448,0.65500,0.3430,0.1030,174.029,4
2,7eYYwvDDg5fBWdYqntL1lI,Angelic Layer,Cruel Choreography,GHOST DATA,32,2021-08-26T07:00:46Z,0.754,0.588,2,-5.993,0,0.0544,0.030900,0.60100,0.1090,0.0902,128.017,4
3,0nJ7Y8j5JoegBoXqHM9pJ6,Ashes,Ashes,Hellberg,30,2021-08-25T07:30:22Z,0.600,0.671,6,-7.666,0,0.0502,0.003880,0.04000,0.1780,0.2970,128.038,4
4,4i5McfiyWkcmVhV8A9oTbv,Last Star,Last Star,James Ivy,40,2021-08-24T22:14:20Z,0.522,0.896,7,-4.901,1,0.0351,0.008870,0.00000,0.2340,0.5050,101.026,4


In [24]:
saved_songs_df.to_pickle("./saved_songs_df.pkl")